In [153]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.pipeline import make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.datasets import make_classification
from sklearn.linear_model import LogisticRegression 
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

**Imputer**

* Для замены `пропусков` в признаках `feat_num` и `feat_cat` используем разные стратегии:
    * `feat_num` - `strategy="mean"`
    * `feat_cat` - `strategy="most_frequent"`
        * 
* strategy="mean" — заполнять пропуски средним по колонке

* strategy="median" — заполнять медианой

* strategy="most_frequent" — заполнять самым частым значением (мода)

* strategy="constant" — заполнять константой (fill_value=...)

In [155]:
# Замена пропусков с помощью SimpleImputer
df = pd.DataFrame({
    "A": [pd.NA, "F", "W", "F", "W"],   # категориальный
    "B": [20, None, 30, 40, 50],         # числовой
    "C": [300, 200, 300, None, pd.NA]    # числовой (с pd.NA)
})

# 1) Заменяем pd.NA -> np.nan
df = df.replace({pd.NA: np.nan})

print('Заменяем pd.NA на np.nan : \n', df)

# TODO Определяем типы колонок
# 2) Импутация: mean для числовых, most_frequent для категориальных
cat_cols = ["A"]
num_cols = ["B", "C"]

# TODO 1-й вариант
# preprocessor = ColumnTransformer(
#     transformers=[
#         ("num", SimpleImputer(strategy="mean"), num_cols),
#         ("cat", SimpleImputer(strategy="most_frequent"), cat_cols)
#     ]
# )


# TODO 2-й вариант
# Импутеры
cat_imputer = SimpleImputer(strategy="most_frequent")
num_imputer = SimpleImputer(strategy="mean")

# Трансформер
preprocessor = ColumnTransformer(
    transformers=[
        ("num", num_imputer, num_cols),
        ("cat", cat_imputer, cat_cols)
    ]
)


# TODO обратно собираем в DF cat и num признаки
result = preprocessor.fit_transform(df)
df_filled = pd.DataFrame(result, columns=num_cols + cat_cols)

print(result)



Заменяем pd.NA на np.nan : 
      A     B      C
0  NaN  20.0  300.0
1    F   NaN  200.0
2    W  30.0  300.0
3    F  40.0    NaN
4    W  50.0    NaN
[[20.0 300.0 'F']
 [35.0 200.0 'F']
 [30.0 300.0 'W']
 [40.0 266.6666666666667 'F']
 [50.0 266.6666666666667 'W']]



**ColumnTransformer ( SimpleImputer() и get_dummies()) - вместе не работают**

**Правила**

* в ColumnTransformer обязательно:
    * все скобки ColumnTransformer `( [ ( имя, трансформатор, столбцы ) ] )`
    * три обязательных компонента `имя, трансформатор, столбцы` должны быть `одним объектом`

In [156]:
df = pd.DataFrame({
    "A": [pd.NA, "F", "W", "F", "W"],   # категориальный
    "B": [20, None, 30, 40, 50],         # числовой
    "C": [300, 200, 300, None, pd.NA]    # числовой (с pd.NA)
})

print('Исходный DF : \n', df, '\n')

# TODO Заменяем pd.NA -> np.nan
data = df.replace({pd.NA: np.nan})

name_col_odj = data.select_dtypes(include=['object', 'category']).columns
name_col_int = data.select_dtypes(include=['int', 'float']).columns

# TODO Трансформаторы для catfeat и numfeat
ohe_obj = OneHotEncoder(handle_unknown="ignore", sparse_output=False)
imput_obj = SimpleImputer(strategy='most_frequent')

scaler_int =StandardScaler()
imput_int = SimpleImputer(strategy='mean')

# TODO pipeline для признаков catfeat и numfeat
pipe_obj = make_pipeline(imput_obj,ohe_obj)
pipe_int = make_pipeline(imput_int, scaler_int)


# TODO SimpleImputer и get_dummies Не Работает
# pipe_obj = make_pipeline(SimpleImputer(strategy='most_frequent'), pd.get_dummies(data=data['A']).astype(int))



data_trsf_obj = ColumnTransformer([('object', pipe_obj, name_col_odj)])
data_trsf_int = ColumnTransformer([('numeric', pipe_int, name_col_int)])


# TODO Имена новых признаков catfeat после ОНЕ и numfeat берём из ColumnTransformer
data_obj = data_trsf_obj.fit_transform(data)
print('Имена новых признаков ОНЕ : \n', data_trsf_obj.get_feature_names_out(), '\n')

data_int = data_trsf_int.fit_transform(data)
print('Имена Стандартизированных признаков int : \n', data_trsf_int.get_feature_names_out(), '\n')

df_obj = pd.DataFrame(data_obj, columns=[data_trsf_obj.get_feature_names_out()])
df_int = pd.DataFrame(data_int, columns=[data_trsf_int.get_feature_names_out()])

data_df = pd.concat([df_int, df_obj], axis=1)

print('Преобразованный DF : ')
data_df

Исходный DF : 
       A     B     C
0  <NA>  20.0   300
1     F   NaN   200
2     W  30.0   300
3     F  40.0  None
4     W  50.0  <NA> 

Имена новых признаков ОНЕ : 
 ['object__A_F' 'object__A_W'] 

Имена Стандартизированных признаков int : 
 ['numeric__B' 'numeric__C'] 

Преобразованный DF : 


numeric__B numeric__C object__A_F object__A_W
0       -1.5   0.912871         1.0         0.0
1        0.0  -1.825742         1.0         0.0
2       -0.5   0.912871         0.0         1.0
3        0.5   0.000000         1.0         0.0
4        1.5   0.000000         0.0         1.0

**pipline DF -> Стандартизация  -> LogReg-модель**


In [154]:
X, y = make_classification(n_samples=1000, n_features=4, n_informative=2, n_redundant=0, n_classes=2, n_clusters_per_class=1 , random_state=42)

print(X.shape, y.shape)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# C = 1 / λ (обратная величина коэффициента регуляризации).
pipe_lg = make_pipeline(StandardScaler(), LogisticRegression(C=10))

pipe_lg.fit(X_train, y_train)
y_pred = pipe_lg.predict(X_test)

print("Accuracy: ", accuracy_score(y_test, y_pred))
print("Precision: ", precision_score(y_test, y_pred, average="macro"))
print("Recall: ", recall_score(y_test, y_pred, average="macro"))
print("F1-score: ", f1_score(y_test, y_pred, average="macro"), '\n')

(1000, 4) (1000,)
Accuracy:  0.9
Precision:  0.9001600640256102
Recall:  0.9
F1-score:  0.8999899989999 

